In [ ]:
# ===============================
# Cell 1 – Install and Setup
# ===============================

!pip install rank-bm25 sentence-transformers spacy faiss-cpu fuzzywuzzy python-Levenshtein --quiet
!python -m spacy download en_core_web_sm --quiet

import json
import os
import numpy as np
from tqdm import tqdm

import spacy
import faiss
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util

# Load spacy for NER
nlp = spacy.load("en_core_web_sm")

# Load sentence embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 116.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# ===============================
# Cell 2 – Mount Drive & Build Page Index
# ===============================

from google.colab import drive

drive.mount('/content/drive')
WIKI_PATH = "/content/drive/MyDrive/wiki-pages"

page_index = {}   # page_id -> file path
title_index = {}  # preprocessed page_id -> page_id

print("Indexing pages...")
for fname in tqdm(os.listdir(WIKI_PATH)):
    if not fname.endswith(".json") and not fname.endswith(".jsonl"):
        continue
    fpath = os.path.join(WIKI_PATH, fname)
    with open(fpath, "r", encoding="utf-8") as f:
        for line in f:
            try:
                data = json.loads(line)
                pid = data["id"]
                page_index[pid] = fpath
                pre_title = pid.replace("_", " ").lower()
                title_index[pre_title] = pid
            except:
                continue

print(f"✅ Indexed {len(page_index)} pages.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Indexing pages...


100%|██████████| 109/109 [02:20<00:00,  1.29s/it]

✅ Indexed 5416537 pages.


In [ ]:
print("Building FAISS IVF index...")

titles_list = list(title_index.keys())
ids_list = [title_index[t] for t in titles_list]

dimension = 384
nlist = 1000
quantizer = faiss.IndexFlatL2(dimension)
ivf_index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

train_limit = min(5000, len(titles_list))
train_embeddings = []

for i in tqdm(range(0, train_limit, 256), desc="Training IVF", unit="batch"):
    batch = titles_list[i:i+256]
    emb = embedder.encode(batch, convert_to_tensor=True).cpu().numpy().astype("float32")
    train_embeddings.append(emb)

train_embeddings = np.concatenate(train_embeddings, axis=0)
ivf_index.train(train_embeddings)
del train_embeddings

for i in tqdm(range(0, len(titles_list), 256), desc="Adding embeddings", unit="batch"):
    batch = titles_list[i:i+256]
    emb = embedder.encode(batch, convert_to_tensor=True).cpu().numpy().astype("float32")
    ivf_index.add(emb)

try:
    res = faiss.StandardGpuResources()
    faiss_index = faiss.index_cpu_to_gpu(res, 0, ivf_index)
    print("✅ FAISS index on GPU.")
except:
    faiss_index = ivf_index
    print("✅ FAISS index on CPU.")

faiss.write_index(faiss_index, "faiss_index_ivf.bin")
print(f"FAISS IVF index built with {len(titles_list)} titles.")


Building FAISS IVF index...


Adding embeddings: 100%|██████████| 21136/21136 [22:17<00:00, 15.81batch/s]


✅ FAISS index on CPU.
FAISS IVF index built with 5410691 titles.


In [ ]:
# ===============================
# Cell 4 – Helpers
# ===============================

def get_page(page_id):
    fpath = page_index.get(page_id, None)
    if not fpath:
        return None
    with open(fpath, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            if data["id"] == page_id:
                return data
    return None

def extract_sentences(page):
    for line in page["lines"].split("\n"):
        if not line.strip():
            continue
        parts = line.split("\t")
        if len(parts) >= 2:
            sent_id, sent_text = parts[0], parts[1]
            yield (sent_text, sent_id)  # generator yields one by one to save RAM


In [ ]:
# ===============================
# Cell 5 – Candidate Retrieval via FAISS
# ===============================

def extract_candidates_with_faiss(query, top_k=5):
    query_emb = embedder.encode(query, convert_to_tensor=True).cpu().numpy().astype('float32').reshape(1, -1)
    distances, indices = faiss_index.search(query_emb, top_k)
    candidates = [ids_list[idx] for idx in indices[0] if idx < len(ids_list)]
    return candidates


In [ ]:
# ===============================
# Cell 6 – Hybrid Retrieval (BM25 + Embeddings)
# ===============================

# def hybrid_retrieval(query, sentences_gen, top_k=3, alpha=0.6):
#     corpus, sent_ids = [], []
#     for sent, sid in sentences_gen:
#         corpus.append(sent)
#         sent_ids.append(sid)

#     if not corpus:
#         return []

#     tokenized_corpus = [doc.split(" ") for doc in corpus]
#     bm25 = BM25Okapi(tokenized_corpus)

#     bm25_scores = bm25.get_scores(query.split(" "))
#     query_emb = embedder.encode(query, convert_to_tensor=True)
#     doc_embs = embedder.encode(corpus, convert_to_tensor=True)
#     cosine_scores = util.cos_sim(query_emb, doc_embs)[0].cpu().numpy()

#     # Normalize
#     bm25_norm = (bm25_scores - bm25_scores.min()) / (bm25_scores.max() - bm25_scores.min() + 1e-8)
#     cos_norm = (cosine_scores - cosine_scores.min()) / (cosine_scores.max() - cosine_scores.min() + 1e-8)
#     final_scores = alpha * cos_norm + (1 - alpha) * bm25_norm

#     ranked = sorted(zip(corpus, final_scores), key=lambda x: x[1], reverse=True)
#     return ranked[:top_k]





In [ ]:
# ===============================
# Cell 7 – Full Retrieval Pipeline
# ===============================

def retrieve_evidence_auto(query, top_k=3, alpha=0.6):
    candidate_titles = extract_candidates_with_faiss(query, top_k=5)
    print(f"Candidate titles for query: {candidate_titles}")

    # Lazy sentence extraction
    all_sentences_gen = (s for title in candidate_titles
                         for s in extract_sentences(get_page(title)))

    return hybrid_retrieval(query, all_sentences_gen, top_k=top_k, alpha=alpha)

In [ ]:
import re

# ===============================
# Example Queries (FEVER style)
# ===============================
queries = [
    "Barack Obama was President of the United States in 2010.",
    "The Berlin Wall fell in 1989.",
    "Titanic was directed by James Cameron.",
    "Mount Everest is the highest mountain on Earth.",
    "Roger Federer has won 20 Grand Slam singles titles."
]
# ===============================
# Retrieval Loop - Improved UI
# ===============================
for idx, q in enumerate(queries, 1):
    print(f"\n===============================")
    print(f"🔎 Query {idx}: {q}")
    print(f"===============================")

    # Retrieve top-3 candidate evidence sentences
    results = retrieve_evidence_auto(q, top_k=5, alpha=0.6)

    for rank, (sent, score) in enumerate(results, 1):
        # Extract possible wiki page refs like [Page_Name]
        pages = re.findall(r"\[([^\]]+)\]", sent)

        # Clean up artifacts in retrieved text
        clean_sent = re.sub(r"\s*--\s*", "–", sent)   # replace -- with –
        clean_sent = clean_sent.replace("-LRB-", "(").replace("-RRB-", ")")

        # Structured display
        print(f"  {rank}. Evidence {'📄' if pages else ''}:")
        print(f"     ➡️ Text : {clean_sent}")
        if pages:
            print(f"     🏷️ Source: {', '.join(pages)}")
        print(f"     🔹 Score : {score:.4f}\n")



🔎 Query 1: Barack Obama was President of the United States in 2010.
Candidate titles for query: ['Barack_Obama_Sr.', 'Presidential_transition_of_Barack_Obama', 'Barack_Obama_presidential_campaign', 'Baracksdubs', 'Baracktrema_obamai']
  1. Evidence :
     ➡️ Text : Barack Obama , the 44th President of the United States , has successfully run for president twice :
     🔹 Score : 0.9925

  2. Evidence :
     ➡️ Text : The Presidential transition of Barack Obama began when Barack Obama won the United States presidential election on November 4 , 2008 , and became the President-elect .
     🔹 Score : 0.8778

  3. Evidence :
     ➡️ Text : Barack Hussein Obama Sr. ( -LSB- ˈbærək_huːˈseɪn_oʊˈbɑːmə -RSB- 18 June 1936–24 November 1982 ) was a Kenyan senior governmental economist and the father of Barack Obama , the 44th President of the United States .
     🔹 Score : 0.8744

  4. Evidence :
     ➡️ Text : Barack Obama presidential campaign , 2008
     🔹 Score : 0.7224

  5. Evidence :
     ➡️ 